In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape, RepeatVector
from keras.layers.merge import concatenate, multiply,add
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

Using TensorFlow backend.


In [29]:
#-------------change according to local directory----------------------

data_dir='../data/'
expert_path='../expert_preds/'
model_path='../KRNN_models/'
checkpoint_dir ='..'


dataset='ECG'  #------can be chosen from ['illness','inpatients','ECG']





In [6]:
def prepareData(ts, label, expert_ts): 
    
    min_in=np.min(ts,axis=1).reshape(ts.shape[0],1)
    max_in=np.max(ts,axis=1).reshape(ts.shape[0],1)
    denom=max_in-min_in
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size>0:
        denom[a]=1
    ts_norm = (ts-min_in)/denom
    expert_ts_norm=(expert_ts-min_in)/denom
    input_x = np.append(ts_norm,expert_ts_norm,axis=1)
#     input_x=ts_norm
    input_y = (label-min_in)/denom
    return input_x.reshape(-1,window_size+horizon,1),input_y,expert_ts_norm,min_in,denom
    
    
def prepareData_2(ts, label, expert_ts): 
    
    min_in=np.min(ts,axis=1).reshape(ts.shape[0],1)
    max_in=np.max(ts,axis=1).reshape(ts.shape[0],1)
    denom=max_in-min_in
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size>0:
        denom[a]=1
    ts_norm = (ts-min_in)/denom
    ts_norm = ts_norm.reshape(-1,window_size,1)
    expert_ts_norm=(expert_ts-min_in)/denom
    
    expert_ts_norm1=np.repeat(expert_ts_norm,window_size)
    expert_ts_norm1= expert_ts_norm1.reshape(-1,window_size,1)
    
    input_x = np.append(ts_norm,expert_ts_norm1,axis=2)

    input_y = (label-min_in)/denom
    return input_x.reshape(-1,window_size,2),input_y,expert_ts_norm,min_in,denom
    
    



def metrics(pred,gt):
    l = pred.shape[1]
#     print(l)
    err_mse = np.zeros((l))
    err_mae = np.zeros((l))

    for i in range(l):
        err_mse[i] = mse(pred[:,i],gt[:,i])
        err_mae[i] = mae(pred[:,i],gt[:,i])
        
    return np.mean(np.sqrt(err_mse)),np.mean(err_mae)
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size), y

def make_k_input(data,horizon):
    length = data.shape[0]
    output= np.zeros([length+1-horizon,horizon])
    for i in range(length-horizon+1):
        output[i:i+1,:]=data[i:i+horizon]
    return output.reshape(output.shape[0],horizon)

def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon

    data = np.append(data,np.zeros([horizon-extra]))

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size), y

def nonov_make_k_input(data,horizon):
    length = data.shape[0]
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],(horizon-extra))
    data = np.append(data,data_app)    

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon):(i*horizon)+horizon]
    return output.reshape(output.shape[0],horizon)

In [67]:

if dataset=='illness':
    window_size = 60
    horizon = 24
    beta=0.001
    data=np.asarray(pd.read_csv(data_dir+'national_illness.csv',usecols=[7]))
    test_length=193
    val_length=77
    train_length=966-test_length-val_length
    
    train=data[:train_length]
    m=np.mean(train)
    s=np.std(train)
    data=(data-m)/s
    train=data[:train_length]    
    val=data[(train_length-window_size):train_length+val_length]



    train_x,train_y = make_input(train[:,0],window_size,horizon)
    p_series_train=np.asarray(pd.read_csv(expert_path+dataset+'/train_h_'+str(horizon)+'.csv',index_col=0))
    p_series_train= (p_series_train-m)/s
    

    val_x,val_y = make_input(val[:,0],window_size,horizon)
    p_series_val=np.asarray(pd.read_csv(expert_path+dataset+'/val_h_'+str(horizon)+'.csv',index_col=0))
    p_series_val= (p_series_val-m)/s
   

    train_in,train_lbl,train_p,_,_=prepareData(train_x,train_y,p_series_train)
    val_in,val_lbl,val_p,_,_=prepareData(val_x,val_y,p_series_val)
    
    
elif dataset =='inpatients':
    window_size = 75
    horizon = 1
    beta=0.01
    #----if monthly--------------
    # test_length=4
    #------if daily---------------
    test_length=28
    data=np.asarray(pd.read_csv(data_dir+dataset+'/inpatients_daily.csv',header=None))
    #------------------------------if monthly-------------------
#     data=np.asarray(pd.read_csv(data_dir+dataset+'/inpatients_monthly.csv',header=None))
    
    train_df = data[0:-test_length]
    test_df = data[(-test_length-window_size):]    
    train_expert = np.asarray(pd.read_csv(expert_path+dataset+'/d_train.csv',usecols=[1])) 
    test_expert = np.asarray(pd.read_csv(expert_path+dataset+'/d_test.csv',usecols=[1]))
    
    #-----------if monthly-----------------
#     train_expert = np.asarray(pd.read_csv(expert_path+dataset+'/m_train.csv'musecols=[1])) 
#     test_expert = np.asarray(pd.read_csv(expert_path+dataset+'/m_test.csv',usecols=[1]))    
    
    train_sequence=make_input(train_df[:,0],window_size)
    test_sequence=make_input(test_df[:,0],window_size)
    train_in, train_lbl, train_p, min_in,denom = prepareData_2(train_sequence[0],train_sequence[1], train_expert)
    test_in, test_lbl, test_p, min_in_test,denom_test = prepareData_2(test_sequence[0], test_sequence[1], test_expert)
    
elif dataset=='ECG':
    window_size = 12
    horizon = 3
    test_length= 498
    train_x=np.zeros((1,window_size+horizon,1))
    train_y=np.zeros((1,horizon))
    test_x=train_x
    test_y=train_y
    p_train=train_y
    output=np.zeros([test_length,140])
    p_test=test_y
    data=np.asarray(pd.read_csv(data_dir+dataset+'/ECG_data.csv',header=None))
    with tqdm(total=140) as pbar:
        for i in range(data.shape[1]):
            train_length=4500
            
            ts=data[:train_length,i]
            ts_x,ts_y = make_input(ts,window_size,horizon)
            p_series_train=np.asarray(pd.read_csv(expert_path+dataset+'/ECG_train/ECG_train_'+str(i+1)+'.csv',index_col=0))
            ts_train,ts_label,ts_p_train,_,_=prepareData(ts_x,ts_y,p_series_train)
            pbar.update(1)




            train_x=np.append(train_x,ts_train,axis=0)
            train_y=np.append(train_y,ts_label,axis=0)
            p_train=np.append(p_train,ts_p_train,axis=0)
    train_lbl=train_y[1:,:]
    train_p=p_train[1:,:]
    train_in=train_x[1:,:]


100%|██████████| 140/140 [00:03<00:00, 35.55it/s]


In [68]:
data.shape

(4998, 140)

In [70]:
#------------------------------------for testing-------------------------------------------

if dataset=='illness':

    model=load_model(model_path+dataset+'/krnn_'+str(horizon)+'_1.h5')
    ts_test=data[-(test_length+window_size):]

    ts_test_x,ts_test_y = nonov_make_input(ts_test[:,0],window_size,horizon)
    p_series_test=(np.asarray(pd.read_csv(expert_path+dataset+'/test_h_'+str(horizon)+'.csv',index_col=0))-m)/s


    ts_test,ts_label,ts_p_test,min_in,denom=prepareData(ts_test_x,ts_test_y,p_series_test)
    preds= model.predict([ts_test,ts_p_test])

    prediction=preds*denom+min_in
    prediction=prediction.flatten()[:test_length]
    
elif dataset=='inpatients':
    model=load_model(model_path+dataset+'/krnn_daily.h5')
    preds = model.predict(([test_X, expert_test]), batch_size=1)
    prediction = (preds*denom_test+min_in_test)

elif dataset=='ECG':
    model=load_model(model_path+dataset+'/krnn.h5')
    output=np.zeros([test_length,140])
    for i in range(data.shape[1]):
        train_length=4500

        ts=data[(train_length-window_size):,i]
        ts_x,ts_y = nonov_make_input(ts,window_size,horizon)
        p_series_test=np.asarray(pd.read_csv(expert_path+dataset+'/ECG_test/ECG_test_'+str(i+1)+'.csv',index_col=0))
        ts_test,ts_label,ts_p_test,min_in,denom=prepareData(ts_x[:,:],ts_y[:,:],p_series_test)


        preds= model.predict([ts_test,ts_p_test])

        prediction=preds*denom+min_in
        prediction=prediction.flatten()[:498]

        output[:,i]=np.transpose(prediction)


In [73]:
if dataset=='ECG':
    temp1, temp2= metrics(output,data[-test_length:,:])
    
    print('RMSE = '+str(np.mean(temp1)))
    print('MAE = '+str(temp2))
else:
    Metrics=[np.sqrt(mse(prediction,data[-test_length:,0])),mae(prediction,data[-test_length:,0])]

    print('RMSE = '+str(Metrics[0]))
    print('MAE = '+str(Metrics[1]))

RMSE = 0.44993350159089496
MAE = 0.32220367374190334


In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 84, 1)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 84, 512)      1052672     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 84, 64)       147712      lstm_1[0][0]                     
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 32)           12416       lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_1 (D

In [30]:
#-----------------For re-training -----------------------------
tf.reset_default_graph()
K.clear_session()

if dataset=='illness':
    inputs_train= Input(batch_shape=(None, window_size+horizon, 1))
    inputs_expert = Input(batch_shape=(None, horizon))


    layer1 = LSTM(512, activation='relu',dropout=0.1, return_sequences=True)(inputs_train)
    layer2 = LSTM(64, activation='relu',dropout=0.1,return_sequences=True)(layer1)
    layer3 = LSTM(32, activation='relu')(layer2)
    layer4 = Dense(horizon,activity_regularizer=regularizers.l2(beta))(layer3)
    residual = add([layer4, inputs_expert])

    model=Model(inputs=[inputs_train, inputs_expert], outputs=residual)
    
    
elif dataset=='inpatients'
    inputs_train= Input(batch_shape=(None,window_size,2))   
    inputs_expert = Input(batch_shape=(None, 1))

    layer1 = Conv1D(16,3, activation='relu')(inputs_train)
    layer2 = Conv1D(8,3, activation='relu')(layer1)  
    layer2=Flatten()(layer2)
    layer2=Dense(1,activity_regularizer=regularizers.l2(beta))(layer2)    
    residual = add([layer2, inputs_expert])
    
    model=Model(inputs=[inputs_train, inputs_expert], outputs=residual)
    
    
elif dataset=='ECG':
    inputs_train= Input(batch_shape=(None, window_size+horizon, 1))    
    inputs_expert = Input(batch_shape=(None, horizon))

    branch_0 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(inputs_train)
    branch_0 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
    layer3=Flatten()(branch_0)
    layer4 = Dense(horizon,activity_regularizer=regularizers.l2(0.0001))(layer3)
    residual = add([layer4, inputs_expert])


    model=Model(inputs=[inputs_train, inputs_expert], outputs=residual)
    

In [173]:
callback = ModelCheckpoint(filepath=checkpoint_dir+'krnn_'+str(horizon)+'_1.h5',monitor='val_loss',save_best_only=True)

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
if dataset=='illness':
    hist=model.fit({'input_1':train_in,'input_2':train_p},train_lbl,validation_data=[[val_in,val_p],val_lbl],callbacks=[callback],batch_size=32,shuffle=False, epochs=100,verbose=1)
else:
    hist=model.fit({'input_1':train_in,'input_2':train_p},train_lbl,validation_split=0.1,callbacks=[callback],batch_size=32,shuffle=False, epochs=100,verbose=1)

Train on 601 samples, validate on 42 samples
Epoch 1/100
601/601 [==============================] - 10s 17ms/step - loss: 0.2300 - val_loss: 0.2982
Epoch 2/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2291 - val_loss: 0.2961
Epoch 3/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2277 - val_loss: 0.2927
Epoch 4/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2260 - val_loss: 0.2883
Epoch 5/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2244 - val_loss: 0.2855
Epoch 6/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2204 - val_loss: 0.2757
Epoch 7/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2220 - val_loss: 0.2812
Epoch 8/100
601/601 [==============================] - 7s 12ms/step - loss: 0.2169 - val_loss: 0.2684
Epoch 9/100
601/601 [==============================] - 7s 12ms/step - loss: 0.3143 - val_loss: 0.2577
Epoch 10/100
601/601 [==============